# 1. Зчитаємо дані

In [33]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head(8)

,oil,petrol,engine,repairs
0,0.875,105,10.3,0
1,0.705,127,10.2,0
2,0.440,144,9.5,1
3,0.350,141,10.7,1
4,0.335,113,9.8,0
5,0.485,139,9.4,1
6,0.755,112,10.1,0
7,0.825,140,9.5,1


In [34]:
df.describe()

,oil,petrol,engine,repairs
count,8.000000,8.000000,8.000000,8.000000
mean,0.596250,127.625000,9.937500,0.500000
std,0.218023,15.583302,0.462717,0.534522
min,0.335000,105.000000,9.400000,0.000000
25%,0.417500,112.750000,9.500000,0.000000
50%,0.595000,133.000000,9.950000,0.500000
75%,0.772500,140.250000,10.225000,1.000000
max,0.875000,144.000000,10.700000,1.000000


# 2. Згенеруємо новий великий датасет для подальшого навчання мережі

In [35]:
import numpy as np
oil_pressure = np.random.normal(loc=df['oil'].mean(), scale=df['oil'].std(), size=1000).tolist()
petrol_lost = np.random.normal(loc=df['petrol'].mean(), scale=df['petrol'].std(), size=1000).tolist()
engine_compression = np.random.normal(loc=df['engine'].mean(), scale=df['engine'].std(), size=1000).tolist()
repairs_need = np.random.choice([0,1], 1000).tolist()

In [36]:
data = {'oil_pressure':oil_pressure,
        'petrol_lost':petrol_lost,
       'engine_compression': engine_compression,
       'repairs_need': repairs_need}
df = pd.DataFrame(data)
df.head()

,oil_pressure,petrol_lost,engine_compression,repairs_need
0,0.100148,119.840818,9.698496,1
1,0.581084,135.727577,9.934148,1
2,0.910040,159.250353,9.497922,1
3,0.978770,130.883206,9.283736,0
4,0.751469,168.216173,10.428437,1


# 3. Препроцесинг данних
1. Для задачі класифікації розділимо датасет на вхідні ознаки (тиск олії, втрати бензину, компресія двигуна), вихідну ознаку - категоріальну змінну, що характеризує необхідність ремонту
2. Вкажемо інтерпретатору, що змінна Y є категоріальною
3. Розіб'ємо датасет на навчальну та тестову вибірки
4. Нормалізуємо зачення

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

scaler = MinMaxScaler()
X = np.array(df[["oil_pressure", "petrol_lost", "engine_compression"]])
Y = np.array(df['repairs_need'])

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# 4. Побудова та навчання мереж
1. Всі мережі ідентичні, проте мають різні значення ваг між нейронами. 
2. Мають по три шари, перший шар має три нейрони, оскільки датасет містить три вхідних ознаки, та функцію активації relu (функція y=x, на проміжку х>0), що дозволяє якнайменше трансформувати вхідні дані. Другий шар чотири нейрони і також функцію активації relu. Останній, вихідний шар, має один нейрон та функцію активації sigmoid, що видає на виході число від 0 до 1, що і є ознакою до якого класу вірогідніше належить вхідний семпл.
3. Мережі мають функцію втрат cross entropy, вихідне значення якої належить проміжку (0, 1), передача цього значення назад (backpropagation) забезпечує навчання мережі. Оптимізатор adam забезпечує змінення значень ваг. Значення accuracy дозволяє побачити ефективність навчання.
4. Етап навчання кожної мережі має 10 епох - 10 повних проходжень навчаального набору даних скрізь мережу. batch_size=1 означає, що датасет подається не відразу повністю, а по одному семплу.
5. Навченна мережа зберігається у .h5 файлі. Значення функції втрат записується у один окремий файл, що дозволить визначити найбільш навченні 10 мереж (чим значення функції втрат, на останій епосі, менше, тим краще навченна мережа)

In [38]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import History 
i = 0
file = open('losses_all.txt', 'w+')
while (i < 25):
    model = Sequential()
    model.add(Dense(3, input_dim=3, activation='relu'))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='mean_squared_error', optimizer='sgd')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics =['accuracy'])
    print('---------new_network_training--------------')
    history = History()
    model.fit(X_train, 
              y_train, 
              epochs=10,
              batch_size=1,
             callbacks=[history])
    model.save(str(i) + '_model.h5')
    file.write(str(i) + '_model:\n' + str(history.history['loss'][9]) + '\n\n')
    del model
    i = i + 1
    
file.close()

---------new_network_training--------------
Epoch 1/10
800/800 [==============================] - 5s 6ms/step - loss: 0.7014 - acc: 0.4962
Epoch 2/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6953 - acc: 0.4875
Epoch 3/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6948 - acc: 0.4912
Epoch 4/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6944 - acc: 0.4775
Epoch 5/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6937 - acc: 0.5075
Epoch 6/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6937 - acc: 0.5112
Epoch 7/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6938 - acc: 0.5013
Epoch 8/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6936 - acc: 0.5075
Epoch 9/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6934 - acc: 0.5125
Epoch 10/10
800/800 [==============================] - 2s 3ms/step - loss: 0.6928 - acc: 0.5138
-----

800/800 [==============================] - 3s 3ms/step - loss: 0.6935 - acc: 0.4800
Epoch 3/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6935 - acc: 0.5038
Epoch 4/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6934 - acc: 0.4862
Epoch 5/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6935 - acc: 0.4825
Epoch 6/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6934 - acc: 0.4800
Epoch 7/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6932 - acc: 0.4912
Epoch 8/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6931 - acc: 0.5112
Epoch 9/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6931 - acc: 0.5000
Epoch 10/10
800/800 [==============================] - 3s 3ms/step - loss: 0.6929 - acc: 0.5175
---------new_network_training--------------
Epoch 1/10
800/800 [==============================] - 5s 7ms/step - loss: 0.6936 - acc: 0.4975
Epoch 2/10
800/8

800/800 [==============================] - 3s 4ms/step - loss: 0.6915 - acc: 0.5337
Epoch 5/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6915 - acc: 0.5400
Epoch 6/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6914 - acc: 0.5250
Epoch 7/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6913 - acc: 0.5200
Epoch 8/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6910 - acc: 0.5325
Epoch 9/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6908 - acc: 0.5225
Epoch 10/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6907 - acc: 0.5463
---------new_network_training--------------
Epoch 1/10
800/800 [==============================] - 6s 7ms/step - loss: 0.6929 - acc: 0.5212
Epoch 2/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6927 - acc: 0.5162
Epoch 3/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6922 - acc: 0.5375
Epoch 4/10
800/8

800/800 [==============================] - 4s 5ms/step - loss: 0.6935 - acc: 0.5050
Epoch 7/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6935 - acc: 0.5000
Epoch 8/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6935 - acc: 0.4912
Epoch 9/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6934 - acc: 0.4837
Epoch 10/10
800/800 [==============================] - 3s 4ms/step - loss: 0.6935 - acc: 0.4900


In [46]:
def from_double_to_categorical(list_of_double):
    encoder = LabelEncoder()
    categorical_list = []
    for elem in list_of_double:
        categorical_elem = 0
        if elem >= 0.5:
            categorical_elem = 1
        
        categorical_list.append(categorical_elem) 
        
    categorical_list = np.array(categorical_list)
    
    return encoder.fit_transform(categorical_list)

# 5. Завантаження та реальна робота мереж
+ порівняння отриманих класів з реально існуючими (y_test)

In [39]:
from keras.models import load_model
model_2 = load_model('2_model.h5')
model_3 = load_model('3_model.h5')
model_6 = load_model('6_model.h5')
model_10 = load_model('10_model.h5')
model_14 = load_model('14_model.h5')
model_15 = load_model('15_model.h5')
model_16 = load_model('16_model.h5')
model_17 = load_model('17_model.h5')
model_18 = load_model('18_model.h5')
model_19 = load_model('19_model.h5')

In [42]:
y_test

array([1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1], dtype=int64)

In [47]:
y_predicted = from_double_to_categorical(model_2.predict(X_test))

In [48]:
y_predicted

array([0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1], dtype=int64)

Міра схожості між двома масивама класів (масив y_test та y_predicted )

In [53]:
import math
from collections import Counter
def counter_cosine_similarity(c1, c2):
    c1 = Counter(c1)
    c2 = Counter(c2)
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB)

In [54]:
counter_cosine_similarity(y_test, y_predicted)

0.9697623183258642

In [55]:
y_predicted = from_double_to_categorical(model_3.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9366942452098452

In [56]:
y_predicted = from_double_to_categorical(model_6.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9487789941274725

In [57]:
y_predicted = from_double_to_categorical(model_10.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9959599955415431

In [58]:
y_predicted = from_double_to_categorical(model_14.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9516452612145582

In [59]:
y_predicted = from_double_to_categorical(model_15.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9873439555792035

In [60]:
y_predicted = from_double_to_categorical(model_16.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9516452612145582

In [61]:
y_predicted = from_double_to_categorical(model_17.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9458488267930967

In [62]:
y_predicted = from_double_to_categorical(model_18.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.9203401368929576

In [63]:
y_predicted = from_double_to_categorical(model_19.predict(X_test))
counter_cosine_similarity(y_test, y_predicted)

0.8806145956889844